# SKRUD Prototype App
> A SKRUD client would _probably_ be implemented in static JS/CSS/HTML. This one uses a kernel per user with `voila`, and implements most of the transformations from the python side 
>> ...for now

In [ ]:
import re, traitlets as T, ipywidgets as W
from tornado import escape, httpclient, ioloop
from pyld import jsonld
from yaml import safe_dump
import random

In [ ]:
RENDERERS = []

In [ ]:
DOC_REL = "http://www.w3.org/ns/hydra/core#apiDocumentation"
CTX_REL = "http://www.w3.org/ns/json-ld#context"
SCM_REL = "describedBy"

In [ ]:
class ScrudClient(W.Textarea):
    value_parsed = T.Dict().tag(sync=True)
    context = T.Dict().tag(sync=True)
    schema = T.Dict().tag(sync=True)
    url = T.Unicode().tag(sync=True)
    doc = T.Dict().tag(sync=True)
    headers = W.trait_types.TypedTuple(
        trait=T.Tuple(T.Unicode(), T.Unicode())
    ).tag(sync=True)
    _client = T.Instance(httpclient.AsyncHTTPClient)
    _token = T.Unicode()

    @T.default("_client")
    def _default_client(self):
        return httpclient.AsyncHTTPClient()
        
    @T.observe("url", "token")
    def _on_url(self, _):
        ioloop.IOLoop.instance().spawn_callback(lambda: self._do_fetch())
    
    def _client_headers(self):
        return {
            "Authorization": f"token {self._token}"
        }
    
    async def _fetch_doc(self, url):
        print(url)
        try:
            response = await self._client.fetch(url, headers=self._client_headers())
        except Exception as err:
            return {"errors": [{"@type": "Error", "description": f"{err}"}]}
        try:
            decoded = escape.json_decode(response.body)
        except Exception as err:
            return {"errors": [{"@type": "Error", "description": f"{err}"}]}
        
        return decoded
        
    async def _do_fetch(self):
        try:
            r = await self._client.fetch(self.url, headers=self._client_headers())
        except Exception as err:
            self.value = safe_dump({"error": f"{err}"})
            return
        
        self.value_parsed = escape.json_decode(r.body)
        self.value = safe_dump(self.value_parsed)
        self.headers = list(r.headers.get_all())
        for match in [
            re.match(r'<(?P<href>.*)>.*rel="(?P<rel>[^"]*)"', link)
            for link in r.headers.get_list("Link")
        ]:
            if match is None:
                return
            groups = match.groupdict()
            rel = groups["rel"]
            href = groups["href"]
            if rel == CTX_REL:
                self.context = await self._fetch_doc(href)
            elif rel == DOC_REL:
                self.doc = await self._fetch_doc(href)
            elif rel == SCM_REL:
                self.schema = await self._fetch_doc(href)

In [ ]:
url = W.Text(
    value="http://localhost:8877/scrud/cpu-temp",
    description="SCRUD URL"
)
layout = dict(rows=10, disabled=True, layout=dict(width="100%"))
token = W.Text(description="Token")
client = ScrudClient(description="Document", **layout)
schema = W.Textarea(description="Schema", **layout)
context = W.Textarea(description="Context", **layout)
headers = W.Textarea(description="Headers", **layout)
doc = W.Textarea(description="API Doc", **layout)
fetch_button = W.Button(description="Fetch")

In [ ]:
T.dlink((client, "schema"), (schema, "value"), safe_dump)
T.dlink((client, "context"), (context, "value"), safe_dump)
T.dlink((client, "headers"), (headers, "value"), safe_dump)
T.dlink((client, "doc"), (doc, "value"), safe_dump)
T.dlink((token, "value"), (client, "_token"))
T.dlink((url, "value"), (client, "url"));

In [ ]:
def _on_click(_):
    client.url = client.url.split("#")[0] + f"#{random.random()}"
fetch_button.on_click(_on_click)

In [ ]:
class WidgetPicker(W.HBox):
    client = T.Instance(ScrudClient, allow_none=True)
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.options = W.SelectMultiple()
        self.widgets = W.VBox()
        self.children = [
            self.options,
            self.widgets
        ]
        self.options.observe(self.options_changed, 'value')
    
    def options_changed(self, change):
        widgets = self.widgets.children
        for widget in widgets:
            del widget
        widgets = []
        for value in self.options.value:
            widgets += [W.HTML(value)]
        self.widgets.children = widgets
    
    @T.observe("client")
    def client_changed(self, change):
        self.client.observe(self.doc_changed, "doc")
    
    def doc_changed(self, change):
        """
        ask the registry for the renderers that are appropriate for where:
        - a renderer registers for one or more `@type`
            - after expanding the resource with its context 
            - and the resource matches one of the fully-expanded types
        - ...
            - use the resource's context to expand the document
            - frame the expanded document with a renderer-provided frame
            - validate the framed document with the renderer's schema
        """
        self.options.options = ["TODO: find widgets", "FIXME: by their frameable type"]
        return
        resource = self.client.value_parsed 
        for renderer in RENDERERS:
            try:
                framed = jsonld.frame(resource, renderer.frame)
                jsonschema.validate(framed, renderer.schema)
                return renderer(framed)
            except:
                pass

In [ ]:
picker = WidgetPicker()
picker.client = client

In [ ]:
details = W.Accordion([
    W.HBox([client, headers]),
    W.HBox([schema, context, doc])
], _titles={0: "Document", 1: "Linked Documents"})

In [ ]:
W.VBox([
    W.HBox([url, token, fetch_button]),
    picker,
    details
])